In [4]:
import re
from datetime import datetime, timedelta
import pandas as pd
from isoweek import Week
#pip install isoweek

In [15]:
def get_hours(iso, peak_type, period):
    # Regular expressions for different period formats
    period_patterns = {
        'annually': r'(\d{4})A',
        'quarterly': r'(\d{4})Q(\d)',
        'monthly': r'(\d{4})(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)',
        'daily': r'(\d{4})-(\d{2})-(\d{2})'
    }

    # Match period with the correct format
    for period_type, pattern in period_patterns.items():
        match = re.match(pattern, period)
        if match:
            break
    else:
        raise ValueError("This is a Wrong Input,\n Input instruction: \n“2018-2-3” as a daily, “2018Mar” as a monthly, “2018Q2” as a quarterly, “2018A” as an annually.")

    year = int(match.group(1))
    

    # Define start and end dates based on period type
    if period_type == 'annually':
        
        start_date = datetime(year, 1, 1)
        end_date = datetime(year, 12, 31)
    elif period_type == 'quarterly':
        quarter = int(match.group(2))
        start_month = 3 * (quarter - 1) + 1
        start_date = datetime(year, start_month, 1)
        end_month = start_month + 2
        end_date = (datetime(year, end_month + 1, 1) - timedelta(days=1))
    #detect month from 3 characters abbreviation
    elif period_type == 'monthly':
        month = datetime.strptime(match.group(2), "%b").month
        start_date = datetime(year, month, 1)
        end_date = (datetime(year, month + 1, 1) - timedelta(days=1))
    elif period_type == 'daily':
        start_date = datetime.strptime(period, "%Y-%m-%d")
        end_date = start_date

    # Process peak type
    include = 0 
    days = pd.date_range(start=start_date, end=end_date, freq='D')

    # Flat Peak for all day
    if peak_type == "flat":
        include = 1
        hours = len(days) * 24
    elif peak_type == "7x8":
        include = 1
        hours = len(days) * 8
    else:
        weekdays = days.weekday
        year_holidays = [Week(year, week).monday() for week in range(1, 53) if Week(year, week).monday().isocalendar()[1] in [1, 52, 53]]
        holidays = [d for d in year_holidays if start_date.date() <= d <= end_date.date()]
        non_holidays = [d for d in days if d not in holidays]

        # Process ISO
        if iso in ["PJMISO", "MISO", "ERCOT", "SPPISO", "NYISO"]:  # Eastern market
            weekend = [5, 6]
        elif iso in ["WECC", "CAISO"]:  # Western market
            weekend = [5]
        else:
            raise ValueError("Wrong input of iso")

        non_holiday_weekdays = [d for d in non_holidays if d.weekday() not in weekend]

        if peak_type == "onpeak":
            hours = len(non_holiday_weekdays) * 16

        elif peak_type == "offpeak":
            include = 1
            hours = len(days) * 24 - len(non_holiday_weekdays) * 16

        elif peak_type == "2x16H":
            weekend_holidays = list(set(holidays).union(set(days[weekdays.isin(weekend)])))
            hours = len(set(weekend_holidays)) * 16

    #Daylight saving
    if iso != "MISO" and include == 1:  # only MISO not participate
        march_dates = pd.date_range(start=datetime(year, 3, 1), end=datetime(year, 3, 31))
        november_dates = pd.date_range(start=datetime(year, 11, 1), end=datetime(year, 11, 30))

        # Find the second Sunday in March and the first Sunday in November
        march_sunday = [d for d in march_dates if d.weekday() == 6][1]
        november_sunday = [d for d in november_dates if d.weekday() == 6][0]

        # If the peak type is 7x8, adjust the number of hours
        if peak_type == "7x8":
            if start_date <= march_sunday <= end_date:
                hours += 1  # Add an hour for the start of daylight saving time
            if start_date <= november_sunday <= end_date:
                hours -= 1  # Subtract an hour for the end of daylight saving time

    # Return the result
    result = {
        "iso": iso,
        "peak_type": peak_type,
        "start_date": start_date,
        "end_date": end_date,
        "num_hour": hours
    }
    return result


## The Daylight Saving Time adjustment of one hour is accounted for in the '7x8' category since it only occurs at 3AM on Sundays.

In [17]:
try1 = get_hours("ERCOT", "offpeak", "2019Mar")
try1

{'iso': 'ERCOT',
 'peak_type': 'offpeak',
 'start_date': datetime.datetime(2019, 3, 1, 0, 0),
 'end_date': datetime.datetime(2019, 3, 31, 0, 0),
 'num_hour': 408}